In [9]:
import json
import pandas as pd
import numpy as np
import plotly as py

In [10]:
df2 = pd.read_json('business_train.json',orient = 'records',lines = True)

In [11]:
ind=[]
cate=df2["categories"].tolist()
for i in range(len(cate)):
    if cate[i]==None:
        continue
    if ("Chinese" in cate[i])|("Thai" in cate[i])|("Japanese" in cate[i])|("Asian" in cate[i])|("Korean" in cate[i]):
        continue
    if ("Brunch" in cate[i])|("Breakfast" in cate[i]):
        ind.append(i)

In [12]:
dfbrunch=df2.iloc[ind]

In [18]:
dfbrunch.index = list(range(len(dfbrunch)))

In [66]:
stars = pd.read_csv('brunch.csv')
stars = stars[['business_id','stars']]

In [79]:
train = stars.groupby(['business_id'])['stars'].agg('mean')

In [69]:
train_new = train.reset_index()[['business_id']]
train_new = train_new.merge(train.reset_index(), suffixes=["", ""], how='left', on=['business_id'])

In [88]:
train_new = train_new.merge(dfbrunch,how = 'left', on = ['business_id'])

In [95]:
train1 = train_new[['business_id','stars','attributes']]

In [110]:
train1.head()

,business_id,stars,attributes
0,133.0,2.944444,"{'BusinessParking': '{'garage': False, 'street..."
1,147.0,4.163121,"{'OutdoorSeating': 'False', 'RestaurantsPriceR..."
2,218.0,4.578512,"{'BikeParking': 'True', 'HasTV': 'False', 'Res..."
3,225.0,3.750000,"{'RestaurantsTakeOut': 'True', 'RestaurantsRes..."
4,469.0,3.620690,"{'BikeParking': 'True', 'RestaurantsDelivery':..."


In [371]:
attribute = ['AcceptsInsurance', 'Alcohol', 'Ambience', 'BYOB','BYOBCorkage', 'BestNights', 'BikeParking','BusinessAcceptsBitcoin',
                 'BusinessAcceptsCreditCards','BusinessParking', 'ByAppointmentOnly', 'Caters', 'CoatCheck','Corkage', 'DietaryRestrictions', 
                'DogsAllowed', 'DriveThru', 'GoodForDancing', 'GoodForKids', 'GoodForMeal', 'HappyHour','HasTV', 'Music', 'NoiseLevel', 
                'Open24Hours', 'OutdoorSeating', 'RestaurantsAttire','RestaurantsCounterService', 'RestaurantsDelivery', 'RestaurantsGoodForGroups',
                'RestaurantsPriceRange2', 'RestaurantsReservations', 'RestaurantsTableService', 'RestaurantsTakeOut', 'Smoking',
                'WheelchairAccessible', 'WiFi']

In [372]:
features = []
for i in range(len(train1)):
    if train1['attributes'][i] != None:
        item = []
        item.append(train1['business_id'][i])
        item.append(train1['stars'][i])
        for j in range(len(attribute)):
            if attribute[j] not in train1['attributes'][i].keys():
                item.append('None')
            else:
                key = attribute[j]
                item.append(str(train1['attributes'][i][key]))
        features.append(item)

In [373]:
feature = pd.DataFrame(features,columns = ['business_id','stars','AcceptsInsurance', 'Alcohol', 'Ambience', 'BYOB','BYOBCorkage', 'BestNights', 'BikeParking','BusinessAcceptsBitcoin',
                 'BusinessAcceptsCreditCards','BusinessParking', 'ByAppointmentOnly', 'Caters', 'CoatCheck','Corkage', 'DietaryRestrictions', 
                'DogsAllowed', 'DriveThru', 'GoodForDancing', 'GoodForKids', 'GoodForMeal', 'HappyHour','HasTV', 'Music', 'NoiseLevel', 
                'Open24Hours', 'OutdoorSeating', 'RestaurantsAttire','RestaurantsCounterService', 'RestaurantsDelivery', 'RestaurantsGoodForGroups',
                'RestaurantsPriceRange2', 'RestaurantsReservations', 'RestaurantsTableService', 'RestaurantsTakeOut', 'Smoking',
                'WheelchairAccessible', 'WiFi'])

In [374]:
feature.head()

,business_id,stars,AcceptsInsurance,Alcohol,Ambience,BYOB,BYOBCorkage,BestNights,BikeParking,BusinessAcceptsBitcoin,...,RestaurantsCounterService,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi
0,133.0,2.944444,None,u'none',"{'romantic': False, 'intimate': False, 'classy...",None,None,None,False,None,...,None,False,True,3,True,None,True,None,None,u'free'
1,147.0,4.163121,None,u'full_bar',"{'romantic': False, 'intimate': False, 'touris...",None,None,"{'monday': False, 'tuesday': False, 'friday': ...",True,None,...,None,False,True,2,False,True,False,u'no',None,u'free'
2,218.0,4.578512,None,'none',"{'romantic': False, 'intimate': False, 'classy...",None,None,None,True,False,...,None,False,True,1,True,False,True,None,True,u'free'
3,225.0,3.750000,None,u'none',None,None,None,None,True,None,...,None,True,None,2,False,None,True,None,True,u'free'
4,469.0,3.620690,None,u'none',"{'romantic': False, 'intimate': False, 'classy...",None,None,None,True,None,...,None,True,True,1,False,False,True,None,None,u'free'


In [313]:
import warnings
warnings.filterwarnings('ignore')
for i in range(len(feature)):
    if feature.Alcohol[i] == "u'none'" or feature.Alcohol[i] == "'none'" or feature.Alcohol[i] == "None":
        feature.Alcohol[i] = 0
    if feature.Alcohol[i] == "u'full_bar'" or feature.Alcohol[i] == "'full_bar'":
        feature.Alcohol[i] = 1
    if feature.Alcohol[i] == "u'beer_and_wine'" or feature.Alcohol[i] == "'beer_and_wine'":
        feature.Alcohol[i] = 2

In [321]:
for i in range(len(feature)):
    if feature.BikeParking[i] == 'False':
        feature.BikeParking[i] = -1
    elif feature.BikeParking[i] == 'None':
        feature.BikeParking[i] = 0
    else:
        feature.BikeParking[i] = 1

In [357]:
for i in range(len(feature)):
    if 'True' in feature.BusinessParking[i]:
        feature.BusinessParking[i] = 'True'
    else:
        feature.BusinessParking[i] = 'False' 

In [358]:
for i in range(len(feature)):
    if feature.NoiseLevel[i] == "u'average'" or feature.NoiseLevel[i] == "'average'":
        feature.NoiseLevel[i] = 'average'
    if feature.NoiseLevel[i] == "u'quiet'" or feature.NoiseLevel[i] == "'quiet'":
        feature.NoiseLevel[i] = 'quiet'
    if feature.NoiseLevel[i] == "u'loud'" or feature.NoiseLevel[i] == "'loud'":
        feature.NoiseLevel[i] = 'loud'
    if feature.NoiseLevel[i] == "u'very_loud'" or feature.NoiseLevel[i] == "'very_loud'":
        feature.NoiseLevel[i] = 'very_loud'        

In [265]:
for i in range(len(feature)):
    if feature.WiFi[i] == "u'free'" or feature.WiFi[i] == "'free'":
        feature.WiFi[i] = 'free'
    if feature.WiFi[i] == "u'no'" or feature.WiFi[i] == "'no'":
        feature.WiFi[i] = 'no'
    if feature.WiFi[i] == "u'paid'" or feature.WiFi[i] == "'paid'":
        feature.WiFi[i] = 'paid'

In [362]:
for i in range(len(feature)):
    feature.stars[i] = int(round(feature.stars[i]))

In [363]:
feature = feature[['stars','Alcohol','BikeParking','BusinessAcceptsCreditCards','BusinessParking','DogsAllowed','GoodForKids',
                         'HasTV','NoiseLevel','OutdoorSeating','RestaurantsDelivery','RestaurantsGoodForGroups','RestaurantsPriceRange2',
                         'RestaurantsReservations','RestaurantsTableService','RestaurantsTakeOut','WiFi']]

In [364]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for col in feature.columns:
    feature[col] = labelencoder.fit_transform(feature[col])
feature.head()

,stars,Alcohol,BikeParking,BusinessAcceptsCreditCards,BusinessParking,DogsAllowed,GoodForKids,HasTV,NoiseLevel,OutdoorSeating,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,WiFi
0,2,0,0,1,1,1,2,0,3,0,0,2,2,2,1,2,4
1,3,1,2,2,1,1,0,0,2,0,0,2,1,0,2,0,4
2,4,0,2,2,1,0,2,0,1,2,0,2,0,2,0,2,4
3,3,0,2,1,0,2,1,0,0,2,2,1,1,0,1,2,4
4,3,0,2,2,1,1,2,0,1,0,2,2,0,0,0,2,4


In [404]:
data = pd.read_csv('attributes.csv')

In [443]:
data.WiFi.value_counts()

 2    2376
-1    1191
 0     682
 1      23
Name: WiFi, dtype: int64

In [441]:
feature.WiFi.value_counts()

u'free'    1601
'free'      775
u'no'       749
None        682
'no'        442
u'paid'      15
'paid'        8
Name: WiFi, dtype: int64

In [405]:
#Alcohol 0:none 1:beer_and_wine 2:full_bar
import warnings
warnings.filterwarnings('ignore')
for i in range(len(data)):
    if data.Alcohol[i] == 1:
        data.Alcohol[i] = 2
    elif data.Alcohol[i] == 2:
        data.Alcohol[i] = 1
    else:
        data.Alcohol[i] = 0

In [406]:
#BikeParking -1:False 0:none 1:True
for i in range(len(data)):
    if data.BikeParking[i] == 1:
        data.BikeParking[i] = 0
    elif data.BikeParking[i] == 0:
        data.BikeParking[i] = -1
    else:
        data.BikeParking[i] = 1

In [407]:
#BusinessAcceptsCreditCards -1:False 0:none 1:True
for i in range(len(data)):
    if data.BusinessAcceptsCreditCards[i] == 1:
        data.BusinessAcceptsCreditCards[i] = 0
    elif data.BusinessAcceptsCreditCards[i] == 0:
        data.BusinessAcceptsCreditCards[i] = -1
    else:
        data.BusinessAcceptsCreditCards[i] = 1

In [408]:
#DogsAllowed -1:False 0:none 1:True
for i in range(len(data)):
    if data.DogsAllowed[i] == 1:
        data.DogsAllowed[i] = 0
    elif data.DogsAllowed[i] == 0:
        data.DogsAllowed[i] = -1
    else:
        data.DogsAllowed[i] = 1

In [409]:
#GoodForKids -1:False 0:none 1:True
for i in range(len(data)):
    if data.GoodForKids[i] == 1:
        data.GoodForKids[i] = 0
    elif data.GoodForKids[i] == 0:
        data.GoodForKids[i] = -1
    else:
        data.GoodForKids[i] = 1

In [410]:
#HasTV -1:False 0:none 1:True
for i in range(len(data)):
    if data.HasTV[i] == 1:
        data.HasTV[i] = 0
    elif data.HasTV[i] == 0:
        data.HasTV[i] = -1
    else:
        data.HasTV[i] = 1

In [414]:
#NoiseLevel  -2:very_loud -1:loud 0:none 1:average 2:quiet
for i in range(len(data)):
    if data.NoiseLevel[i] == 1:
        data.NoiseLevel[i] = 1
    elif data.NoiseLevel[i] == 0:
        data.NoiseLevel[i] = 0
    elif data.NoiseLevel[i] == 4:
        data.NoiseLevel[i] = -2
    elif data.NoiseLevel[i] == 2:
        data.NoiseLevel[i] = -1
    else:
         data.NoiseLevel[i] = 2

In [418]:
#OutdoorSeating -1:False 0:none 1:True
for i in range(len(data)):
    if data.OutdoorSeating[i] == 1:
        data.OutdoorSeating[i] = 0
    elif data.OutdoorSeating[i] == 0:
        data.OutdoorSeating[i] = -1
    else:
        data.OutdoorSeating[i] = 1

In [421]:
#RestaurantsDelivery  -1:False 0:none 1:True
for i in range(len(data)):
    if data.RestaurantsDelivery[i] == 1:
        data.RestaurantsDelivery[i] = 0
    elif data.RestaurantsDelivery[i] == 0:
        data.RestaurantsDelivery[i] = -1
    else:
        data.RestaurantsDelivery[i] = 1

In [424]:
#RestaurantsGoodForGroups  -1:False 0:none 1:True
for i in range(len(data)):
    if data.RestaurantsGoodForGroups[i] == 1:
        data.RestaurantsGoodForGroups[i] = 0
    elif data.RestaurantsGoodForGroups[i] == 0:
        data.RestaurantsGoodForGroups[i] = -1
    else:
        data.RestaurantsGoodForGroups[i] = 1

In [429]:
#RestaurantsPriceRange2
for i in range(len(data)):
    if data.RestaurantsPriceRange2[i] == 1:
        data.RestaurantsPriceRange2[i] = 2
    elif data.RestaurantsPriceRange2[i] == 0:
        data.RestaurantsPriceRange2[i] = 1
    elif data.RestaurantsPriceRange2[i] == 4:
        data.RestaurantsPriceRange2[i] = 0
    elif data.RestaurantsPriceRange2[i] == 2:
        data.RestaurantsPriceRange2[i] = 3
    else:
         data.RestaurantsPriceRange2[i] = 4

In [432]:
#RestaurantsReservations -1:False 0:none 1:True
for i in range(len(data)):
    if data.RestaurantsReservations[i] == 1:
        data.RestaurantsReservations[i] = 0
    elif data.RestaurantsReservations[i] == 0:
        data.RestaurantsReservations[i] = -1
    else:
        data.RestaurantsReservations[i] = 1

In [435]:
#RestaurantsTableService -1:False 0:none 1:True
for i in range(len(data)):
    if data.RestaurantsTableService[i] == 1:
        data.RestaurantsTableService[i] = 0
    elif data.RestaurantsTableService[i] == 0:
        data.RestaurantsTableService[i] = -1
    else:
        data.RestaurantsTableService[i] = 1

In [439]:
#RestaurantsTakeOut  -1:False 0:none 1:True
for i in range(len(data)):
    if data.RestaurantsTakeOut[i] == 1:
        data.RestaurantsTakeOut[i] = 0
    elif data.RestaurantsTakeOut[i] == 0:
        data.RestaurantsTakeOut[i] = -1
    else:
        data.RestaurantsTakeOut[i] = 1

In [442]:
#WiFi
for i in range(len(data)):
    if data.WiFi[i] == 4 or data.WiFi[i] == 0:
        data.WiFi[i] = 2
    elif data.WiFi[i] == 5 or data.WiFi[i] == 1:
        data.WiFi[i] = -1
    elif data.WiFi[i] == 3:
        data.WiFi[i] = 0
    else:
         data.WiFi[i] = 1

In [445]:
data['stars'] = feature['stars']